 **To**: Megaline Management

 **From**: Junior Data Scientist

 **Date**: August 8, 2025
 
 **Subject**: Preliminary Analysis of Surf and Ultimate Prepaid Plans

# Introduction
This report presents a preliminary analysis of Megaline's 'Surf' and 'Ultimate' prepaid plans. The primary goal is to determine which of these two plans generates more revenue. By analyzing the behavior of a sample of 500 customers from 2018, we can gain insights that will help the commercial department make informed decisions about the allocation of the advertising budget.